In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import cv2
import csv
import h5py
import random
import matplotlib.pyplot as plt

# Extraction every 5 seconds

In [ ]:
data_l = np.load('/bigpool/Team3/OneSec/P10_OneSec_Label_Final.npz')
label = data_l['label_person_np']
data_g = np.load('/bigpool/Team3/OneSec/P10_OneSec_Gaze30_Final.npz')
gaze = data_g['gaze_person_np']

#comparison of time stamps
label_time = label[:,0]
gaze_time = gaze[:,1,0]
print(np.array_equal(label_time, gaze_time))

# taking samples every 5 second instead of every second --> label and gaze
(n,) = label[:,0].shape

m = n/5
r = n%5
if r == 0:
    mi = int(m)
else:
    mi = int(m) + 1

in_prep = np.arange(0,mi)
index = in_prep * 5

label_5sec = np.take(label,index,axis = 0)
gaze_5sec = np.take(gaze,index,axis = 0)

np.savez('/bigpool/Team3/FiveSec/P10_FiveSec_Label.npz',label_5sec = label_5sec)
np.savez('/bigpool/Team3/P10_FiveSec_Gaze_New.npz',gaze_5sec = gaze_5sec)

In [1]:
# taking samples every 5 second instead of every second --> for frame numbers and timestamps of images --> maintained just for tracking

data_tf = np.load('/bigpool/Team3/P10_Timestamp_Frames_Both.npz')
time_stamp_frame = data_tf['total_timestamp']
time_stamp_frame = time_stamp_frame.astype(int)
(n,) = time_stamp_frame[:,0].shape
m = n/5
r = n%5
if r == 0:
    mi = int(m)
else:
    mi = int(m) + 1
    
in_prep = np.arange(0,mi)
index = in_prep * 5

time_frame_both_5sec = np.take(time_stamp_frame,index,axis = 0)

np.savez('/bigpool/Team3/P10_FiveSec_TimeStamps_Frmaes_Both.npz',time_frame_both_5sec = time_frame_both_5sec)


# taking samples every 5 second instead of every second --> for images of scene information

data_v = np.load('/bigpool/Team3/P04_OneSec_Frames.npz')
video = data_v['input1']

n = video.shape[0]

m = n/5
r = n%5
if r == 0:
    mi = int(m)
else:
    mi = int(m) + 1

in_prep = np.arange(0,mi)
index = in_prep * 5  

video_5sec = np.take(video,index,axis = 0)

np.savez('/bigpool/Team3/P04_FiveSec_Frames.npz',video_5sec = video_5sec)

# HDF5 creation

In [2]:
DATA_PATH = "./FiveSec/"

In [ ]:
hf = h5py.File(DATA_PATH + 'data.h5', 'a')
for s in ["Training_Set", "Validation_Set", "Testing_Set"]:
    if (not(s in hf.keys() )):
        x = hf.create_group(s)#create separate group for training , testing and validation set
    else:
        x = hf[s]

In [ ]:
for p in range(1, 11):     
    if ((p == 3) | (p == 8)):
        Set = "Testing_Set"
    elif (p == 6):
        Set = "Validation_Set"
    else:
        Set = "Training_Set"
        
    print("%sP%02d_FiveSec_Label.npz" % (DATA_PATH, p,))
    label = np.load("%sP%02d_FiveSec_Label.npz" % (DATA_PATH, p,))
    if ('Total_Label' in hf[Set].keys()):
        hf[Set + '/Total_Label'].resize((hf[Set + '/Total_Label'].shape[0] + label['label_5sec'].shape[0]), axis = 0)
        hf[Set + '/Total_Label'][-label['label_5sec'].shape[0]:] = label['label_5sec']
    else:        
        print("New Total_Label in %s"  % Set)
        hf[Set].create_dataset("Total_Label", data= label['label_5sec'], maxshape = (None,10)) #create dataset for labels
    label.close()

    print("%sP%02d_FiveSec_Gaze.npz" % (DATA_PATH, p,))
    gaze = np.load("%sP%02d_FiveSec_Gaze.npz" % (DATA_PATH, p,))
    if ('Total_Gaze' in hf[Set].keys()):
        hf[Set + '/Total_Gaze'].resize((hf[Set + '/Total_Gaze'].shape[0] + gaze['gaze_5sec'].shape[0]), axis = 0)
        hf[Set + '/Total_Gaze'][-gaze['gaze_5sec'].shape[0]:] = gaze['gaze_5sec']
    else:
        print("New Total_Gaze in %s"  % Set)
        hf[Set].create_dataset("Total_Gaze", data= gaze['gaze_5sec'], maxshape = (None,30,9)) #create dataset for gaze
    gaze.close()

    print("%sP%02d_FiveSec_Frames.npz" % (DATA_PATH, p,))
    video = np.load("%sP%02d_FiveSec_Frames.npz" % (DATA_PATH, p,))
    frames = video['video_5sec']
    lst = []
    size = np.array(frames.shape) // 2
    for i in range(len(frames)):
        lst.append(cv2.resize(frames[i], (size[2], size[1]), interpolation=cv2.INTER_LANCZOS4))
    imgs = np.concatenate(lst)
    imgs = imgs.reshape(-1, size[1],  size[2], 3)
    
    if ('Total_Video' in hf[Set].keys()):
        hf[Set + '/Total_Video'].resize((hf[Set + '/Total_Video'].shape[0] + imgs.shape[0]), axis = 0)
        hf[Set + '/Total_Video'][-imgs.shape[0]:] = imgs
    else:
        print("New Total_Video in %s"  % Set)
        hf[Set].create_dataset("Total_Video", data=imgs, maxshape = (None, size[1],  size[2], 3))#create dataset for frames
    del imgs
    del frames
    video.close()

# Extraction every 10 seconds

In [ ]:
HDF5_PATH = "/bigpool/Team3/FiveSec/data.h5"

hdf = h5py.File(HDF5_PATH, "r")
#data with samples every 5 second (inclusion of augmented data)
Train_v = hdf['Validation_Set/Total_Video']
Train_g = hdf['Validation_Set/Total_Gaze']
Train_l = hdf['Validation_Set/Total_Label']

# taking samples every 10 second instead of every 5 second 
n = Train_v.shape[0]
m = n/2
r = n%2
if r == 0:
    mi = int(m)
else:
    mi = int(m) + 1
    
print(m)
print(mi)
in_prep = np.arange(0,mi)
index = in_prep * 2
video_10sec = (np.take(Train_v,index,axis = 0))/255 #/255 is for normalization of the images
gaze_10sec = np.take(Train_g,index,axis = 0)
label_10sec = np.take(Train_l,index,axis = 0)

video_10sec = video_10sec.astype(np.float32)# convertion to float32 from float64

np.savez("S_Val/S_Validation_video_10sec_norm.npz",video_10sec = video_10sec)
np.savez("S_Val/S_Validation_gaze_10sec.npz",gaze_10sec = gaze_10sec)
np.savez("S_Val/S_Validation_label_10sec.npz",label_10sec = label_10sec)

#same code can be slightly modified (like chanGing the file directory) to prepare data for Testing and Training